In [1]:
from collections import defaultdict
import numpy as np
from pyspark.ml.linalg import SparseVector
from pyspark.sql.functions import explode
from pyspark import SparkFiles
from pyspark.sql import Row

import bz2
import json
import time
from pyspark.ml import Pipeline
from pyspark.ml.feature import * # CountVectorizer, Tokenizer, RegexTokenizer, HashingTF
from pyspark.ml.regression import * # RandomForestRegressor, LinearRegression, DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1607373263861_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
def timeit(method):
    '''
    Decorator to time functions.
    '''
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        print('%r took %2.2f sec\n' % (method.__name__, te-ts))
              
        return result
    return timed

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import re
# DATAFILE_PATTERN = '^(.+),"(.+)",(.*),(.*),(.*)'
ID_PATTERN = '"id":(.*?(?=,|}))'
UPS_PATTERN = '"ups":(.*?(?=,|}))'
BODY_PATTERN = '"body":(.*?(?=,|}))'
# DOWNS_PATTERN = '"downs":(.*?(?=,|}))'
SCORE_PATTERN = '"score":(.*?(?=,|}))'
# CONTROVERSIALITY_PATTERN = '"controversiality":(.*?(?=,|}))'

def removeQuotes(s):
    """ Remove quotation marks from an input string
    Args:
        s (str): input string that might have the quote "" characters
    Returns:
        str: a string without the quote characters
    """
    return ''.join(i for i in s if i!='"')

def parseDatafileLine(datafileLine):
    """ Parse a line of the data file using the specified regular expression pattern
    Args:
        datafileLine (str): input string that is a line from the data file
    Returns:
        tuple: a tuple including the parsed results using the given regular expression and without the quote characters
    """
    id_match = re.search(ID_PATTERN, datafileLine.decode('utf-8'))
    ups_match = re.search(UPS_PATTERN, datafileLine.decode('utf-8'))
    body_match = re.search(BODY_PATTERN, datafileLine.decode('utf-8'))
    score_match = re.search(SCORE_PATTERN, datafileLine.decode('utf-8'))
    
    if (id_match is None) or (ups_match is None) or (body_match is None) or (score_match is None):
        print('Invalid datafile line: %s' % datafileLine)
        return (datafileLine, -1)
    else:
        viralness = 0
        if int(score_match.group(1)) < -10 or int(score_match.group(1)) > 10:
            viralness = 1
        comment = (id_match.group(1), int(ups_match.group(1)), removeQuotes(body_match.group(1)), int(score_match.group(1)), viralness)
        return (comment, 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import sys
import os
from pyspark import SparkFiles

#RC_PATH = '/FileStore/shared_uploads/ddk1@andrew.cmu.edu/RC_2007_10'
RC_PATH = 's3://dsml-vasu-simar-daniel/RC_2015-0*'

def parseData(path):
    """ Parse a data file
    Args:
        filename (str): input file name of the data file
    Returns:
        RDD: a RDD of parsed lines
    """
#     sc.addFile(path)
    return (sc
            .textFile(path, 4, 0)
            .map(parseDatafileLine)
            .cache())
@timeit
def loadData(path):
    """ Load a data file
    Args:
        path (str): input file name of the data file
    Returns:
        RDD: a RDD of parsed valid lines
    """

    raw = parseData(path).cache()
    
    failed = (raw
              .filter(lambda s: s[1] == -1)
              .map(lambda s: s[0]))
    for line in failed.take(10):
        print('%s - Invalid datafile line: %s' % (path, line))
    
    deleted = (raw
             .filter(lambda s: s[0][2] == '[deleted]')
             .map(lambda s: s[0]))
    
    valid = (raw
             .filter(lambda s: s[1] == 1)
             .filter(lambda s: s[0][2] != '[deleted]')
             .map(lambda s: s[0])
             .cache())
   
    viral = (raw
             .filter(lambda s: s[1] == 1)
             .filter(lambda s: s[0][4] == 1)
             .map(lambda s: s[0])
             .cache())
    nonviral = (raw
               .filter(lambda s: s[1] == 1)
               .filter(lambda s: s[0][4] == 0)
               .map(lambda s: s[0])
               .cache())
    nonviral_cut = nonviral.sample(False, viral.count()/nonviral.count())
    print(nonviral.count())
    nonviral_cut.take(10)
    viral_nonviral_cut = viral.union(nonviral_cut)
    viral_nonviral_cut.take(10)
    
    print('%s - Read %d lines, successfully parsed %d lines, failed to parse %d lines, %d lines were deleted, %d lines were viral, %d lines were non-viral, %d viral and non-viral lines were returned' % (path,
                                                                                                                                                                                                 raw.count(),
                                                                                                                                                                                                 valid.count(),
                                                                                                                                                                                                 failed.count(),
                                                                                                                                                                                                 deleted.count(),
                                                                                                                                                                                                 viral.count(),
                                                                                                                                                                                                 nonviral.count(),
                                                                                                                                                                                                 viral_nonviral_cut.count()))                                                  
    return viral_nonviral_cut

reddit = loadData(RC_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

145084211
s3://dsml-vasu-simar-daniel/RC_2015-0* - Read 156758730 lines, successfully parsed 147697364 lines, failed to parse 0 lines, 9061366 lines were deleted, 11674519 lines were viral, 145084211 lines were non-viral, 23346486 viral and non-viral lines were returned
'loadData' took 238.10 sec

In [6]:
sentenceDF = reddit.toDF().selectExpr("_1 as id", "_2 as ups", "_3 as body", "_4 as score", "_5 as viralness")
sentenceDF.show(n=5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---+--------------------+-----+---------+
|       id|ups|                body|score|viralness|
+---------+---+--------------------+-----+---------+
|"cnas8zv"| 14|Most of us have s...|   14|        1|
|"cnas90g"| 17|&gt;&gt;If a woma...|   17|        1|
|"cnas91f"| 19|Made a multiscree...|   19|        1|
|"cnas91m"| 41|A lot of disrespe...|   41|        1|
|"cnas91w"| 14|You hear that SDS...|   14|        1|
+---------+---+--------------------+-----+---------+
only showing top 5 rows

In [7]:
split_regex = r'\W+'
linebreak_regex = r'\\r\\n\\r\\n'

def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    linebreak_removed_string = re.sub(linebreak_regex, " ", string)
    return list(filter(None, re.split(split_regex, linebreak_removed_string.lower())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
stopwords = set(sc.textFile("s3://dsml-vasu-simar-daniel/stopwords.txt").collect())
print('These are the stopwords: %s' % stopwords)

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    return list(filter(lambda word: word not in stopwords,simpleTokenize(string)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

These are the stopwords: {'out', 'we', 'was', 'how', 'myself', 'for', 'they', 'about', 'then', 'both', 'so', 'don', 'as', 'any', 'after', 'you', 'why', 'been', 'where', 'by', 'yourself', 'a', 'did', 'their', 'doing', 'be', 'further', 'ours', 'now', 'am', 'her', 'yourselves', 'that', 'what', 'my', 'to', 'not', 'own', 'there', 'this', 'each', 'all', 'more', 'me', 'which', 'himself', 'nor', 'other', 'who', 'same', 'at', 'such', 't', 'up', 'than', 'can', 'too', 'these', 'while', 'before', 'ourselves', 'he', 'i', 'our', 'its', 'but', 'with', 'because', 'those', 'the', 'it', 'hers', 'just', 'over', 'between', 'had', 'does', 'have', 'and', 'some', 'or', 'only', 'when', 'below', 'in', 'if', 'theirs', 'again', 'his', 'whom', 'above', 'should', 'itself', 'themselves', 'until', 'are', 'she', 'will', 'from', 'into', 'no', 'your', 'few', 'herself', 'of', 'has', 'down', 'were', 'once', 'having', 'them', 'under', 'him', 'do', 'on', 'an', 'yours', 'being', 'off', 'very', 'through', 'most', 'against', 

In [9]:
redditRecToToken = reddit.map(lambda line: (line[0], line[1], tokenize(line[2]),line[3], line[4]))

print(redditRecToToken.take(5))

def countTokens(vendorRDD):
    """ Count and return the number of tokens
    Args:
        vendorRDD (RDD of (recordId, tokenizedValue)): Pair tuple of record ID to tokenized output
    Returns:
        count: count of all tokens
    """
    # TODO: Uncomment the template below and replace <FILL IN> with appropriate code
    recordCount = vendorRDD.map(lambda line: len(line[0]))
    recordSum = recordCount.sum()
    return recordSum

totalTokens = countTokens(redditRecToToken)
print('There are %s tokens in the combined datasets' % totalTokens)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('"cnas8zv"', 14, ['us', 'family', 'members', 'like', 'family', 'like'], 14, 1), ('"cnas90g"', 17, ['gt', 'gt', 'woman', 'wants', 'give', 'child', 'adoption', 'named', 'father'], 17, 1), ('"cnas91f"', 19, ['made', 'multiscreen', 'video', 'sony', 'vegas', 'converted', 'webm'], 19, 1), ('"cnas91m"', 41, ['lot', 'disrespecting', 'league', 'today', 'think', 'll', 'eventually', 'need', 'see', 'garber', 'publicly', 'reprimanding', 'nycfc', 'ownership', 'group', 'doubt', 'would', 'happen', 'though'], 41, 1), ('"cnas91w"', 14, ['hear', 'sdss', 'j122952', '66', '112227', '8', 'heard', 'talking', 'shit', 're', 'coming', 'ya'], 14, 1)]
There are 210118374 tokens in the combined datasets

In [10]:
@timeit
def term_frequency(df, inputCol, outputCol, hashFeatures=None):
    '''
    Returns a DataFrame object containing a new row with the extracted features. 
    Passing hashed=True will return a Featured Hashed matrix.
    
    @params:
        df - DataFrame
        inputCol - name of input column from DataFrame to find features
        outputCol - name of the column to save the features
        hashFeatures - number of features for HashingTF, if None will perform 
            CountVectorization
    '''
    
    # since the number of features was not passed perform standard CountVectorization
    if hashFeatures is None:
        cv = CountVectorizer(inputCol=inputCol, outputCol=outputCol)
        feature_extractor = cv.fit(df)
    # otherwise perform a feature extractor with 
    else:
        feature_extractor = HashingTF(\
                              inputCol=inputCol, outputCol=outputCol, numFeatures=hashFeatures)
    
    # create a new DataFrame using either feature extraction method
    return feature_extractor.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
wordsFilteredDF = spark.createDataFrame(redditRecToToken).toDF("id", "ups", "filtered_words", "score", "viralness")

# Feature Hash the comment content
# number of features for Feature Hash matrix, reccomended too use power of 2
hashDF = term_frequency(\
    df=wordsFilteredDF, inputCol="filtered_words", outputCol="features", hashFeatures=2048)

# Display snippet of new DataFrame
hashDF.select('filtered_words','features').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'term_frequency' took 0.31 sec

+--------------------+--------------------+
|      filtered_words|            features|
+--------------------+--------------------+
|[us, family, memb...|(2048,[455,929,13...|
|[gt, gt, woman, w...|(2048,[159,782,80...|
|[made, multiscree...|(2048,[15,474,689...|
|[lot, disrespecti...|(2048,[85,159,396...|
|[hear, sdss, j122...|(2048,[122,340,58...|
+--------------------+--------------------+
only showing top 5 rows

In [12]:
@timeit
def random_forest_regression(df, featuresCol, labelCol):
    '''
    Returns a DataFrame containing a column of predicted values of the labelCol.
    Predict the output of labelCol using values in featuresCol y = rf(x).
    
    @params:
        df - DataFrame
        featuresCol - input features, x
        labelCol - output variable, y
    '''
    # split the training and test data using the holdout method
    (trainingData, testData) = df.randomSplit([0.8, 0.2])
    
    # create the random forest regressor, limit number of trees to ten
    dtr = RandomForestRegressor(\
       featuresCol=featuresCol, labelCol=labelCol)
    
    # fit the training data to the regressor to create the model
    model = dtr.fit(trainingData)
    
    # create a DataFrame contained a column with predicted values of the labelCol
    predictions = model.transform(testData)
    
    return predictions

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# train random forest regression
rfpredictions = random_forest_regression(df=hashDF,featuresCol="features",labelCol="viralness")

# compute the error
evaluator = RegressionEvaluator(labelCol="viralness", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'random_forest_regression' took 648.53 sec

Root Mean Squared Error (RMSE) on test data = 0.495928

In [14]:
rfpredictions.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---+--------------------+-----+---------+--------------------+-------------------+
|       id|ups|      filtered_words|score|viralness|            features|         prediction|
+---------+---+--------------------+-----+---------+--------------------+-------------------+
|"cnas90g"| 17|[gt, gt, woman, w...|   17|        1|(2048,[159,782,80...|  0.557443813240796|
|"cnas91w"| 14|[hear, sdss, j122...|   14|        1|(2048,[122,340,58...| 0.5194807409847766|
|"cnas953"| 24|[league, cup, con...|   24|        1|(2048,[213,758,90...| 0.5043114595821483|
|"cnas97r"| 17|[unpopular, opinion]|   17|        1|(2048,[298,636],[...| 0.5043114595821483|
|"cnas989"| 14|                  []|   14|        1|        (2048,[],[])| 0.5043114595821483|
|"cnas98l"| 13|[really, strange,...|   13|        1|(2048,[14,526,180...| 0.5043114595821483|
|"cnas98y"| 13|          [fairness]|   13|        1|  (2048,[260],[1.0])| 0.5043114595821483|
|"cnas9bm"| 26|        [holy, shit]|   26|        1|(2048,[5

In [15]:
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
@timeit
def logistic_regression_viral(df, featuresCol, labelCol):
    '''
    Returns a DataFrame containing a column of predicted values of the labelCol.
    Predict the output of labelCol using values in featuresCol y = rf(x).
    
    @params:
        df - DataFrame
        featuresCol - input features, x
        labelCol - output variable, y
    '''
    # split the training and test data using the holdout method
#     print(df.count())
#     df = df.filter((df.score >=0) & (df.score <10))
#     print(df.show(10))
#     print(df.count())
    (trainingData, testData) = df.randomSplit([0.8, 0.2])
    
    
    # TODO: Uncomment the lines below and replace <FILL IN> with appropriate code
    # Given hyperparameters
    standardization = False
    elastic_net_param = 0.8
    reg_param = .3
    max_iter = 10

    lr = (LogisticRegression(featuresCol=featuresCol, labelCol=labelCol, regParam = reg_param, standardization = standardization, maxIter = max_iter,elasticNetParam = elastic_net_param))
  
    lr_model_basic = lr.fit(trainingData)

    trainingSummary = lr_model_basic.summary
    accuracy = trainingSummary.accuracy
    print(accuracy)
    # create a DataFrame contained a column with predicted values of the labelCol
    predictions = lr_model_basic.transform(testData)
    
    return predictions

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# train random forest regression
lrViralpredictions = logistic_regression_viral(df=hashDF,featuresCol="features",labelCol="viralness")

# compute the error
evaluator = RegressionEvaluator(labelCol="viralness", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lrViralpredictions)
print ("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5000063174460986
'logistic_regression_viral' took 116.95 sec

Root Mean Squared Error (RMSE) on test data = 0.706931

In [17]:
lrViralpredictions.show(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
|       id| ups|      filtered_words|score|viralness|            features|       rawPrediction|         probability|prediction|
+---------+----+--------------------+-----+---------+--------------------+--------------------+--------------------+----------+
|"cnas91w"|  14|[hear, sdss, j122...|   14|        1|(2048,[122,340,58...|[-2.5269784395451...|[0.49999368255390...|       1.0|
|"cnas93e"|  22|[oh, man, n, nhtt...|   22|        1|(2048,[16,342,391...|[-2.5269784395451...|[0.49999368255390...|       1.0|
|"cnas96o"|  86|      [bron, person]|   86|        1|(2048,[365,505],[...|[-2.5269784395451...|[0.49999368255390...|       1.0|
|"cnas97m"|  69|[option, 0, tell,...|   69|        1|(2048,[35,414,445...|[-2.5269784395451...|[0.49999368255390...|       1.0|
|"cnas98l"|  13|[really, strange,...|   13|        1|(2048,[14,526,180...|[-2.5269784395451...|[0.499993

In [18]:
hashDF = hashDF.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# train random forest regression
lrViralpredictions = logistic_regression_viral(df=hashDF,featuresCol="features",labelCol="viralness")

# compute the error
evaluator = RegressionEvaluator(labelCol="viralness", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(lrViralpredictions)
print ("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5000267970357178
'logistic_regression_viral' took 52.89 sec

Root Mean Squared Error (RMSE) on test data = 0.706989

In [20]:
# train random forest regression
rfpredictions = random_forest_regression(df=hashDF,featuresCol="features",labelCol="viralness")

# compute the error
evaluator = RegressionEvaluator(labelCol="viralness", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rfpredictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'random_forest_regression' took 323.39 sec

Root Mean Squared Error (RMSE) on test data = 0.495968